In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline

In [17]:
data=pd.read_csv('/home/anu/Downloads/sms_spam_ham_collection',sep='\t',header=None)
data

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [19]:
lb=LabelBinarizer()
pipe = Pipeline([('vect',TfidfVectorizer(stop_words='english')),('est',LogisticRegression())])
parameters={
    'vect__max_df':(0.25,0.5,0.75),
    'vect__stop_words':([None]),
    'vect__max_features':(2500,5000,10000,None),
    'vect__ngram_range':((1,1),(1,2)),
    'vect__use_idf':(True,False),
    'vect__norm':('l1','l2'),
    'est__penalty':('l1','l2'),
    'est__C':(0.01,0.1,1,10)
}
# est_c => lambda value
#vect__max_features=>(2500,5000,10000,None)#can take either 1st 2500 values,or 5000 or none
grid_search=GridSearchCV(pipe,parameters,n_jobs=-1,\
                        scoring='accuracy',cv=3,verbose=1)
xtrain,xtest,ytrain,ytest=train_test_split(data.iloc[:,1],data.iloc[:,0])
ytrain=np.array([num[0] for num in lb.fit_transform(ytrain)])
grid_search.fit(xtrain,ytrain)

Fitting 3 folds for each of 768 candidates, totalling 2304 fits


/home/anu/yes/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        na

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(stop_words='english')),
                                       ('est', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'est__C': (0.01, 0.1, 1, 10),
                         'est__penalty': ('l1', 'l2'),
                         'vect__max_df': (0.25, 0.5, 0.75),
                         'vect__max_features': (2500, 5000, 10000, None),
                         'vect__ngram_range': ((1, 1), (1, 2)),
                         'vect__norm': ('l1', 'l2'), 'vect__stop_words': [None],
                         'vect__use_idf': (True, False)},
             scoring='accuracy', verbose=1)

In [20]:
print('Best score:%2f'%grid_search.best_score_)

Best score:0.983967


In [21]:
print(grid_search.best_params_)

{'est__C': 10, 'est__penalty': 'l2', 'vect__max_df': 0.5, 'vect__max_features': 2500, 'vect__ngram_range': (1, 2), 'vect__norm': 'l2', 'vect__stop_words': None, 'vect__use_idf': True}


In [22]:
best_parameters=grid_search.best_estimator_.get_params()

for param_name in sorted(parameters.keys()):
    print('%s: %r'%(param_name,best_parameters[param_name]))

est__C: 10
est__penalty: 'l2'
vect__max_df: 0.5
vect__max_features: 2500
vect__ngram_range: (1, 2)
vect__norm: 'l2'
vect__stop_words: None
vect__use_idf: True


In [23]:
y_test=np.array([num[0] for num in lb.fit_transform(ytest)])
predictions=grid_search.predict(xtest)

In [24]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

print('Accuracy score:',accuracy_score(y_test,predictions))
print('Precision score:',precision_score(y_test,predictions))
print('Recall score:',recall_score(y_test,predictions))
print('F1 score:',f1_score(y_test,predictions))



Accuracy score: 0.9834888729361091
Precision score: 0.9776536312849162
Recall score: 0.9020618556701031
F1 score: 0.938337801608579
